<a href="https://colab.research.google.com/github/blackrosedragon2/Text-Summarization/blob/master/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install tqdm
!pip install pyspellchecker


     |████████████████████████████████| 1.9MB 2.8MB/s 


In [2]:
#Reading data onto zipfile object ZF
from google.colab import drive
import os 
import zipfile
drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/data/'  
zf=zipfile.ZipFile(root_path+"cnn_stories.zip")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import re
from spellchecker import SpellChecker
class preprocess:
    #Accepts a string and performs string operations on it
    def seperate(self,para):#1
        # find @highlight, add length and split story and summary, then decode to utf-8 format, use split tokenizer and seperate the 4 lines , split the \n parts and return the list
        story=para[:para.find('@highlight')]
        summary=[i.splitlines()[2] for i in (para[para.find('@highlight')+len('@highlight'):]).split('@highlight')]
        return story,summary    
        #returns (string,list)
    
    def spell_check(self,para):#2
        #corrects spellings(may not be used for us) + takes much longer to correct spellings (5+ minutes)
        spell = SpellChecker()
        para=para.split(' ')
        correctpara=[]
        for word in para:
            correctpara.append(spell.correction(word))
        return ' '.join(correctpara)

    def remove_apostrophes(self,para):#3
        return re.sub("(?<=[a-z])'(?=[a-z])", "", para)

    def remove_parentheses(self,para):#4
        #removes anything inside parentheses
        return re.sub(r'\([^)]*\)', '', para)

    def remove_sp(self,para):#5
        #removes special symbols
        return re.sub('[^A-Za-z0-9]+', ' ', para)   #note: add exception for words like U.N. 

    def remove_short(self,para):#6
        #removes words that are <3 in length
        para=para.split(' ')
        longpara=[]
        for word in para:
            if len(word) >= 3:
                longpara.append(word)
        return ' '.join(longpara)

    def add_tags(self,para):#7
        #add start and end tags 
        para="<start>"+para+"<end>"
        return para

    def apply_all(self,para):#8
        #applies 1,3,4,5,6 functions, may need reordering 
        p=preprocess()

        story,summary=p.seperate(para.decode("utf-8"))

        #story=p.spell_check(story)
        #summary=p.spell_check(summary)

        story=p.remove_apostrophes(story)
        summary=[p.remove_apostrophes(i) for i in summary] 

        story=p.remove_parentheses(story)
        summary=[p.remove_parentheses(i) for i in summary]

        story=p.remove_sp(story)
        summary=[p.remove_sp(i) for i in summary]

        story=p.remove_short(story)
        summary=[p.remove_short(i) for i in summary]

        return story,summary



In [46]:
#Read content of all files onto x (takes approximatly 13 minutes for both modes)
from tqdm import tqdm

def get_data(preproc=0):
    i=1
    if preproc==1:
        stories=[]
        story=[]
        summaries=[]
        summary=[]
        p=preprocess()
        with tqdm(total=92600) as pbar:
            while(True):
                #Reads zip file until eof exception occurs
                i=i+1
                try:
                    story,summary=p.apply_all(zf.open(zf.namelist()[i]).read())
                    stories.append(story)
                    summaries.append(summary)
                except Exception as E:
                    break
                pbar.update(1)
        return stories,summaries
    else:
        x=[]
        with tqdm(total=92600) as pbar:
            while(True):
                #Reads zip file until eof exception occurs
                i=i+1
                try:
                    x.append(zf.open(zf.namelist()[i]).read())
                except Exception as E:
                    break
                pbar.update(1)
        return x

_,summa=get_data(1)


100%|█████████▉| 92579/92600 [12:04<00:00, 127.71it/s]


In [47]:
for j,i in enumerate(summa):
    print(i)
    if j>10:
        break

['Syrian official Obama climbed the top the tree doesnt know how get down', 'Obama sends letter the heads the House and Senate', 'Obama seek congressional approval military action against Syria', 'Aim determine whether were used not whom says spokesman']
['Usain Bolt wins third gold world championship', 'Anchors Jamaica 4x100m relay victory', 'Eighth gold the championships for Bolt', 'Jamaica double womens 4x100m relay']
['The employee agencys Kansas City office among hundreds virtual workers', 'The employees travel and from the mainland last year cost more than 000', 'The telecommuting program like all GSA practices under review']
['NEW Canadian doctor says she was part team examining Harry Burkhart 2010', 'NEW Diagnosis autism severe anxiety post traumatic stress disorder and depression', 'Burkhart also suspected German arson probe officials say', 'Prosecutors believe the German national set string fires Los Angeles']
['Another arrest made gang rape outside California school', 'Inves